# Projet maison n° 4

In [23]:
# imports
import requests
from bs4 import BeautifulSoup
import re

**Partie A**

Ecrire une fonction `get_prices_from_url()` qui extrait des informations à partir des 2 pages ci-dessous.

```python
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"
```

Avec `URL_PAGE2`, la fonction doit retourner :

```json
{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}
```

In [24]:
# partie A
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"

def get_prices_from_url(url):
    prices = {}
    r1 = requests.get(url)
    content1 = r1.content.decode('utf-8')
    soup1 = BeautifulSoup(content1)
    Plans = soup1.find_all('div', {'class': "pricing-table"})
    for plan in Plans:
        key = plan.find('h2').text
        price = re.search('[$][0-9]+',plan.find('span').text)[0]
        prices[key] = {'price':price}
        storage = re.search('[0-9]+.B',plan.text)[0]
        prices[key]['storage'] = storage
        databases = re.findall('[0-9]+',re.search('[0-9]+ database',plan.text)[0])[0]
        prices[key]['databases'] = int(databases)
    return prices

**Partie B**

*L'abus d'alcool est dangereux pour la santé, à consommer avec modération.*

1) Ecrire une fonction `extract_beer_infos()` qui extrait des informations sur une bière du site de bières *beowulf*.

Exemple d'URL: https://www.beerwulf.com/fr-fr/p/bieres/cuvee-des-trolls.33 

La fonction doit retourner :
```json
{'Nom': 'Cuvée des Trolls',
 'Style': 'Bière Blonde',
 'Contenu': 25,
 'Degré d’alcool': 7.0,
 'Origine': 'Belgique',
 'Brasseur': 'Brasserie Dubuisson Freres'}
```

2) L'URL ci-après retourne un JSON avec une liste de bières :

```python
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"
```

Ecrire une fonction `extract_beer_list_infos(url)` qui prend en argument cet URL et retourne les informations sur une liste de bières du site *beowulf*.

Cette fonction doit retourner la liste des informations obtenues par la fonction précédemment définie `extract_beer_infos()`.

Exemple de retour :

```json
[{'Nom':'Desperados','Style':'Lager','Contenu':33,'Degré d’alcool':5.9,'Origine':'France','Brasseur':'Desperados'},
{'Nom':'La Lager Sans Gluten de Vézelay','Style':'Lager','Contenu':25,'Degré d’alcool':4.0,'Origine':'France','Brasseur':'Brasserie de Vézelay'},
{'Nom':'Mélusine Bio','Style':'Pale Ale','Contenu': 33,'Degré d’alcool': 5.0,'Origine':'France','Brasseur':'Mélusine'},
{'Nom':'La Parisienne Le Titi Parisien','Style':'IPA','Contenu':33,'Degré d’alcool':5.5,'Origine':'France','Brasseur': 'Brasserie la Parisienne'},
{'Nom':'Brasserie De Sutter Brin de Folie','Style':'Bière Blonde','Contenu': 33,'Degré d’alcool':6.5,'Origine':'France','Brasseur':'Brasserie de Sutter'}]
```

**Facultatif**

Chercher comment optimiser cette fonction en utilisant `multiprocessing.Pool()` pour paralléliser les accès web.

In [46]:
# partie B-1
def extract_beer_infos(url):
    beer_infos = {
        'Nom': None,
        'Style': None,
        'Contenu': None,
        'Degré d’alcool': None,
        'Origine': None,
        'Brasseur': None
    }
    r1 = requests.get(url)
    content1 = r1.content.decode('utf-8')
    soup1 = BeautifulSoup(content1)
    #nom = re.search('(.*)(?= -)',soup1.find('h1', {'class': "product-name"}).text)[0]
    soup1.find('h1', {'class': "product-name"}).text
    style = soup1.find('dl', {'class': "row"}).find('a').text
    contenu = int(re.search('[0-9,]+',soup1.find('dl', {'class': "row"}).find('dd', {'class': "small-6 medium-9 columns js-beer-volume"}).text)[0])
    degre = float(re.search('[0-9,]+%',soup1.find('dl', {'class': "row"}).text)[0][:-1].replace(',','.'))
    origine = re.search('[A-Za-z]+',soup1.find('dl', {'class': "row"}).find('dd', {'class': "small-6 medium-9 columns js-beer-country"}).text)[0]
    brasseur = soup1.find('dl', {'class': "row"}).find_all('a')[1].text
    beer_infos['Nom'] = nom
    beer_infos['Style'] = style
    beer_infos['Contenu'] = contenu
    beer_infos['Degré d’alcool'] = degre
    beer_infos['Origine'] = origine
    beer_infos['Brasseur'] = brasseur
    return beer_infos

In [26]:
# partie B-2
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"

def extract_beer_list_infos(url):
        
    # Collecter les pages de bières à partir du JSON
    beer_pages = []
    r2 = requests.get(URL_BEERLIST_FRANCE)
    for item in r2.json()['items']:
        beer_url = 'https://www.beerwulf.com' + item['contentReference']
        beer_pages.append(beer_url)
    
    # Sequential version (slow):
    beers = []
    for beer in beer_pages:
        beers.append(extract_beer_infos(beer))

    # Facultatif
    # Parallel version (faster):
    # beers = []
    
    return beers

In [27]:
import unittest

class Session4Tests(unittest.TestCase):
    def test_01_get_prices_from_url_page2(self):
        prices = get_prices_from_url(URL_PAGE2)
        # We should have found 3 products:
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 3)
        self.assertIn('Personal', prices)
        self.assertIn('Small Business', prices)
        self.assertIn('Enterprise', prices)
        
        personal = prices['Personal']
        self.assertIn('price', personal)
        self.assertIn('storage', personal)
        self.assertIn('databases', personal)
        self.assertEqual(personal['price'], '$5')
        self.assertEqual(personal['storage'], '1GB')
        self.assertEqual(personal['databases'], 1)
        
    def test_02_get_prices_from_url_page3(self):
        prices = get_prices_from_url(URL_PAGE3)
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 4)
        self.assertEqual(
            prices['Privilege'],
            {'databases': 100, 'price': '$99', 'storage': '1TB'}
        )
    
    def test_03_extract_beer_list_infos(self):
        infos = extract_beer_list_infos(URL_BEERLIST_FRANCE)
        self.assertIsInstance(infos, list)
        self.assertGreater(len(infos), 1)
        # Contenu = int
        # Degré d’alcool = float
        for beer in infos:
            self.assertIsInstance(beer['Nom'], str)
            self.assertIsInstance(beer['Style'], str)
            self.assertIsInstance(beer['Contenu'], int)
            self.assertIsInstance(beer['Degré d’alcool'], float)
            self.assertEqual(beer['Origine'], "France")
            self.assertIsInstance(beer['Brasseur'], str)

            
def run_tests():
    test_suite = unittest.makeSuite(Session4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [47]:
if __name__ == '__main__':
    run_tests()

test_01_get_prices_from_url_page2 (__main__.Session4Tests) ... ok
test_02_get_prices_from_url_page3 (__main__.Session4Tests) ... ok
test_03_extract_beer_list_infos (__main__.Session4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 13.672s

OK
